In [1]:
import numpy as np
import pandas as pd
import os
import glob
import codecs
import scispacy
import spacy
import sklearn
import warnings 

from collections import defaultdict, Counter
from sklearn.model_selection import KFold
# from negspacy.negation import Negex
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import defaultdict
from transformers import BertTokenizer
from nltk.corpus import stopwords

Neither PyTorch nor TensorFlow >= 2.0 have been found.Models won't be available and only tokenizers, configurationand file/data utilities can be used.


In [2]:

  
warnings.filterwarnings(action = 'ignore') 
  
# print(model1.similarity("Hospice", "Immunology"))                               
# nlp0 = spacy.load("en_core_web_sm")
# negex = Negex(nlp0, language = "en_clinical")
# nlp0.add_pipe(negex, last=True)
# doc0 = nlp0("he is not an emotional eater")
import string

#string punctuation
punct = set(string.punctuation)
# for e in doc0.ents:
# 	print(e.text, e._.negex)

# Using scispacy with "en_core_sci_sm":
nlp = spacy.load("en_core_sci_sm")

# # Using scispacy with "en_core_sci_lg":
# nlp = spacy.load("en_core_sci_lg")

# # Using scispacy with "en_ner_bc5cdr_md:
# nlp = spacy.load("en_ner_bc5cdr_md")


def tokenize_doc(text):
    bow = defaultdict(float)
    temp = []
    doc = nlp(text)
    doc = doc.ents
    lowered_tokens = map(lambda t: str(t).lower(), doc)

    for token in lowered_tokens:
        # token = token.split(".,")
        # if (len(token) > 1):
        #     for j in token:
        #         j = j.split(":,")
        #         for i in j:
        #             bow[i] += 1.0
        #             temp.append(i)
        # else:
        #     for i in token:
        #         bow[i] += 1.0
        #         temp.append(i)

        temp.append(token)
    return temp

def tokenize_doc2(text):
    t = BertTokenizer.from_pretrained("bert-large-uncased-vocab.txt", lowercase=True)
    warnings.filterwarnings('ignore')
    output = t.tokenize(text)
    temp = []
    words = [w for w in output if w not in punct]
    for token in words:
        temp.append(token)
    return temp

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data = pd.read_csv("mtsamples.csv")

In [ ]:
ids = data["transcription"]
data = data.drop_duplicates(subset = ["description","medical_specialty","sample_name","transcription"])
temp = [i for i in data.index if pd.isna(data["transcription"][i])]
data = data.drop(temp)
print(len(data))
        

In [ ]:
data_list = []

for i in range(len(data["transcription"])):
    try:
        data_list.append(data["transcription"][i])
    except:
        print(i)


In [ ]:
print(len(data_list))

In [ ]:
data = data.reset_index(drop=True)

In [ ]:
print(data['transcription'][0])

In [ ]:

test = tokenize_doc2(data['transcription'][0])   
print(test) 

In [ ]:
print(tokenize_doc(data['transcription'][0])  )

In [ ]:
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer


kf = KFold(n_splits=12, random_state=None, shuffle=False)
kf.get_n_splits(data)

Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_doc)
# Tfidf_vect = TfidfVectorizer(tokenizer=tokenize_doc2)
features  = Tfidf_vect.fit(data['transcription'])


In [ ]:
from sklearn.svm import NuSVC
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

nsvc = NuSVC(nu=0.0001, probability=True)
lsvc = LinearSVC(tol = 1e-5, C = 0.0001, random_state=500000)

In [ ]:
count = 0
total = 0
res = {}
for train_index, test_index in kf.split(data):
    # print("TRAIN:", len(train_index), "TEST:", len(test_index))
    # for train_index, test_index in kf.split(data):
    x_train = []
    y_train = []

    x_train, x_test = data['transcription'][train_index], data['transcription'][test_index]
    y_train, y_test = data['medical_specialty'][train_index], data['medical_specialty'][test_index]

    Train_X_Tfidf = Tfidf_vect.transform(x_train)
    Test_X_Tfidf = Tfidf_vect.transform(x_test)
    
#     nsvc.fit(Train_X_Tfidf,y_train) 
    lsvc.fit(Train_X_Tfidf,y_train) 
    
#     predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
    predictions_nuSVM = lsvc.predict(Test_X_Tfidf)
    
    print(metrics.classification_report(predictions_nuSVM, y_test.to_numpy(), digits=5))
    total += accuracy_score(y_test.to_numpy(), predictions_nuSVM)*100
    count += 1 
    res[count] = (predictions_nuSVM, y_test.to_numpy())


    print(accuracy_score(y_test, predictions_nuSVM)*100)
#     total += accuracy_score(y_test, predictions_nuSVM)*100
#     count += 1

# print(count)
# print(total/count)
print(total/count)
print(res)
    


In [ ]:
x_train, x_test = data['transcription'][0:5], data['transcription'][0:5]
y_train, y_test = data['medical_specialty'][0:5], data['medical_specialty'][0:5]
                                                  
Train_X_Tfidf = Tfidf_vect.transform(x_train)
Test_X_Tfidf = Tfidf_vect.transform(x_test)

nsvc.fit(Train_X_Tfidf,y_train)   
#classifier2.fit(Train_X_Tfidf,y_train)

predictions_nuSVM = nsvc.predict(Test_X_Tfidf)
print(accuracy_score(predictions_nuSVM, y_test)*100)

In [ ]:
print(len(Tfidf_vect.vocabulary_))

In [ ]:
print(Tfidf_vect.vocabulary_)

In [ ]:
print(Tfidf_vect.get_feature_names())